In [268]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tqdm import tqdm_notebook
import lightgbm as lgb
from catboost import Pool, CatBoostClassifier

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns



plt.style.use('seaborn')
sns.set(font_scale=1)

import gc

In [269]:
train = pd.read_csv("../input/cmu-africa-data-science-club-challenge/train (9).csv")
test = pd.read_csv("../input/cmu-africa-data-science-club-challenge/test (10).csv")
sample = pd.read_csv('../input/cmu-africa-data-science-club-challenge/sample_submission_fixed.csv')

In [270]:
id_=test["id"]
test.drop("id", axis=1, inplace=True)

In [271]:
cols=['REGION-CENTROID-COL', 'REGION-CENTROID-ROW', 'REGION-PIXEL-COUNT',
       'SHORT-LINE-DENSITY-5', 'SHORT-LINE-DENSITY-2', 'VEDGE-MEAN',
       'VEDGE-SD', 'HEDGE-MEAN', 'HEDGE-SD', 'INTENSITY-MEAN', 'RAWRED-MEAN',
       'RAWBLUE-MEAN', 'RAWGREEN-MEAN', 'EXRED-MEAN', 'EXBLUE-MEAN',
       'EXGREEN-MEAN', 'VALUE-MEAN', 'SATURATION-MEAN', 'HUE-MEAN']

In [272]:
# y_train = train[['CLASS_0','CLASS_1', 'CLASS_2', 'CLASS_3', 'CLASS_4', 'CLASS_5', 'CLASS_6']]
y_train = train[['CLASS']]
df_train=train.drop(['CLASS'], axis=1)

In [275]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=5168)
# oof = df_train[['ID_code', 'target']]
# oof['predict'] = 0
# predictions = test[['ID_code']]
# feature_importance_df = pd.DataFrame()
# val_aucs = []
for fold, (trn_idx, val_idx) in enumerate(skf.split(df_train, y_train)):
    X_train, y_train = df_train.iloc[trn_idx][cols], train.iloc[trn_idx]['CLASS']
    X_valid, y_valid = df_train.iloc[val_idx][cols], train.iloc[val_idx]['CLASS']
    break
    
    
    
clf = CatBoostClassifier(loss_function = "MultiClass", eval_metric = "AUC",random_seed=123,use_best_model=True,
                          learning_rate=0.01,  iterations=25000,verbose=100,
                           bootstrap_type= "Poisson", 
                           task_type="GPU", 
#                              l2_leaf_reg= 16.5056753964314982, depth= 3.0,
#                              fold_len_multiplier= 2.9772639036842174, 
#                              scale_pos_weight= 3.542962442406767, 
#                              fold_permutation_block_size=16.0, subsample= 0.46893530376570957
#                              fold_len_multiplier=3.2685541035861747, 
#                              scale_pos_weight= 2.6496926337120916, 
#                              fold_permutation_block_size= 6.0, 
                          )
print("Model training")
clf.fit(X_train, y_train,  eval_set=(X_valid, y_valid), early_stopping_rounds=2000,verbose=100)

Model training
0:	test: 0.9521429	best: 0.9521429 (0)	total: 15ms	remaining: 6m 15s


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


100:	test: 0.9980952	best: 0.9985714 (26)	total: 1.14s	remaining: 4m 40s
200:	test: 0.9980952	best: 0.9985714 (26)	total: 2.27s	remaining: 4m 40s
300:	test: 0.9980952	best: 0.9985714 (26)	total: 3.4s	remaining: 4m 39s
400:	test: 0.9985714	best: 0.9985714 (26)	total: 4.9s	remaining: 5m
500:	test: 0.9985714	best: 0.9985714 (26)	total: 6.11s	remaining: 4m 58s
600:	test: 0.9985714	best: 0.9985714 (26)	total: 7.25s	remaining: 4m 54s
700:	test: 0.9985714	best: 0.9985714 (26)	total: 8.43s	remaining: 4m 52s
800:	test: 0.9985714	best: 0.9985714 (26)	total: 9.58s	remaining: 4m 49s
900:	test: 0.9985714	best: 0.9985714 (26)	total: 10.7s	remaining: 4m 46s
1000:	test: 0.9985714	best: 0.9985714 (26)	total: 11.9s	remaining: 4m 44s
1100:	test: 0.9985714	best: 0.9985714 (26)	total: 13s	remaining: 4m 42s
1200:	test: 0.9985714	best: 0.9985714 (26)	total: 14.2s	remaining: 4m 40s
1300:	test: 0.9985714	best: 0.9985714 (26)	total: 15.5s	remaining: 4m 42s
1400:	test: 0.9985714	best: 0.9985714 (26)	total: 16.8s

In [276]:
cols=test.columns

In [277]:
predict = clf.predict_proba(test[cols])

In [278]:
sub=pd.DataFrame({"id":id_,"CLASS_0":predict[:,0] ,"CLASS_1":predict[:,1],"CLASS_2":predict[:,2],"CLASS_3":predict[:,3],"CLASS_4":predict[:,4],"CLASS_5":predict[:,5],"CLASS_6":predict[:,6]})

In [279]:
sub

,id,CLASS_0,CLASS_1,CLASS_2,CLASS_3,CLASS_4,CLASS_5,CLASS_6
0,0,0.143188,0.142913,0.188026,0.131005,0.131008,0.130995,0.132866
1,1,0.132530,0.133329,0.132536,0.132539,0.198770,0.135534,0.134762
2,2,0.131410,0.130521,0.128393,0.222804,0.128403,0.128403,0.130066
3,3,0.137723,0.131200,0.206876,0.130273,0.130272,0.130270,0.133387
4,4,0.129598,0.130198,0.128885,0.220524,0.128886,0.128886,0.133023
...,...,...,...,...,...,...,...,...
2095,2095,0.142824,0.131548,0.209028,0.127735,0.127738,0.127732,0.133396
2096,2096,0.138598,0.139192,0.131796,0.131721,0.130876,0.130574,0.197243
2097,2097,0.129634,0.128847,0.128461,0.220835,0.128470,0.128470,0.135283
2098,2098,0.129247,0.128178,0.128704,0.128181,0.128181,0.229101,0.128409


In [280]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")


In [281]:
create_submission(sub, "sol")

/kaggle/working/sol.csv